# CAE-Transformer流场预测模型

## 环境安装

本案例要求 **MindSpore >= 2.0.0** 版本以调用如下接口: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*。具体请查看[MindSpore安装](https://www.mindspore.cn/install)。

此外，你需要安装 **MindFlow >=0.1.0** 版本。如果当前环境还没有安装，请按照下列方式选择后端和版本进行安装。

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## 概述

### 背景介绍

降阶模型可有效降低使用CFD方法的设计成本和周期。对于复杂的可压缩流动，使用POD等线性方法进行流场降维，需要大量的模态才能保证流场重建的精度，而采用非线性降维方法能够有效减少所需模态数。卷积自编码器(CAE)是一种由编码器和解码器组成的神经网络，能够实现数据降维和重构，可看作是POD方法的非线性拓展。采用CAE进行流场数据的非线性降维，同时使用Transformer进行流场状态的时间演化。对于非定常可压缩流动，“CAE-Transformer”降阶模型能够在使用较少自由变量数与较短计算周期的前提下获得较高的重构和预测精度。

### 模型架构

CAE-Transformer的基本框架主要基于[论文1](https://doi.org/10.13700/j.bh.1001-5965.2022.0085)和[论文2](https://doi.org/10.1609/aaai.v35i12.17325)，其由CAE和Transformer组成，其中CAE中的编码器降低时间序列流场的维数，实现特征提取，Transformer学习低维时空特征并进行预测，CAE中的解码器实现流场重建：

+ 输入：输入一段时间的流场；

+ 压缩：通过CAE的编码器对流场进行降维，提取高维时空流动特征；

+ 演化：通过Transformer学习低维空间流场时空特征的演变，预测下一时刻；

+ 重建：通过CAE的解码器将预测的流场低维特征恢复到高维空间；

+ 输出：输出对下一时刻瞬态流场的预测结果。

![CAE-Transformer1.png](./images/cae_transformer_structure.png)

### 数据集

来源：二维圆柱绕流数值仿真流场数据，由北京航空航天大学航空科学与工程学院于剑副教授团队提供。

生成方法：二维圆柱绕流的数据集计算状态与建立方法见论文[Ma Z, Yu J, Xiao R. Data-driven reduced order modeling for parametrized time-dependent flow problems[J]. Physics of Fluids, 2022, 34(7).](https://pubs.aip.org/aip/pof/article/34/7/075109/2847227/Data-driven-reduced-order-modeling-for)

数据说明：数据集针对10个雷诺数的圆柱绕流进行了数值模拟，每个雷诺数下的流场数据包含401个时间步，每个时间步的流场数据为256*256的二维流场，每个变量的数据类型为float32，数据集总大小为约1.96GB。
数据集下载链接:

[2D_cylinder_flow.npy](https://download.mindspore.cn/mindscience/mindflow/dataset/applications/data_driven/cae-transformer/2D_cylinder_flow.npy)

## 准备工作

导入训练所需函数库，其中src文件夹包括数据集处理函数、网络模型和训练loss可视化函数。

训练默认采用Mindspore框架的动态图模式(PYNATIVE)，在GPU(默认)或Ascend进行训练(单卡)。

In [ ]:
import os
import time
import argparse
import yaml
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, jit, data_sink
from src import create_caetransformer_dataset, plot_train_loss, CAE_Informer
from eval import cae_transformer_prediction, cae_transformer_eval

np.random.seed(0)
set_seed(0)
parser = argparse.ArgumentParser(description="CAE-Transformer for 2D cylinder flow")
parser.add_argument(
    "--mode",
    type=str,
    default="PYNATIVE",
    choices=["GRAPH", "PYNATIVE"],
    help="Context mode, support 'GRAPH', 'PYNATIVE'",
)
parser.add_argument(
    "--save_graphs",
    type=bool,
    default=False,
    choices=[True, False],
    help="Whether to save intermediate compilation graphs",
)
parser.add_argument("--save_graphs_path", type=str, default="./graphs")
parser.add_argument(
    "--device_target",
    type=str,
    default="GPU",
    choices=["GPU", "Ascend"],
    help="The target device to run, support 'Ascend', 'GPU'",
)
parser.add_argument(
    "--device_id", type=int, default=0, help="ID of the target device"
)
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args()
context.set_context(mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs,
                    save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target,
                    device_id=args.device_id)

## 读取参数

从`config.yaml`文件里导入相应的数据集、CAE模型和优化器的参数配置。

In [ ]:
config_file_path = "./config.yaml"
# prepare params
with open(config_file_path, 'r') as f:
    config = yaml.safe_load(f)
data_params = config["data"]
model_params = config["cae_transformer"]
optimizer_params = config["optimizer"]

# prepare summary file
summary_dir = optimizer_params["summary_dir"]
ckpt_dir = os.path.join(summary_dir, "ckpt")

## 模型初始化

根据config.yaml中的配置，初始化模型，包括CAE和Transformer网络。使用MSELoss损失函数和Adam优化器。

In [ ]:
# prepare model
model = CAE_Informer(**model_params)
loss_fn = nn.MSELoss()
optimizer = nn.AdamWeightDecay(
    model.trainable_params(),
    optimizer_params["lr"],
    weight_decay=optimizer_params["weight_decay"],
)

def forward_fn(data, label):
    logits = model(data)
    loss = loss_fn(logits, label)
    return loss

time_now = time.time()
model.set_train()

grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=False)

@jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    loss = ops.depend(loss, optimizer(grads))
    return loss

## 数据集构建

根据config.yaml中读取的数据路径构建CAE-Transforme数据集，并做数据下沉。

数据集链接为: [2D_cylinder_flow.npy](https://download.mindspore.cn/mindscience/mindflow/dataset/applications/data_driven/cae-transformer/2D_cylinder_flow.npy)

In [ ]:
# prepare dataset
dataset, eval_data = create_caetransformer_dataset(
    data_params['data_path'],
    data_params["batch_size"],
    data_params["seq_len"],
    data_params["pred_len"],
)

# data sink
sink_process = data_sink(train_step, dataset, sink_size=1)
train_data_size = dataset.get_dataset_size()


## 模型训练

使用MindSpore >= 2.0.0的版本，可以使用函数式编程范式训练神经网络。

In [ ]:
print(f"====================Start cae transformer train=======================")
train_loss = []
model.set_train()
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(model, f"{ckpt_dir}/model_{epoch}.ckpt")
    if epoch % optimizer_params["eval_interval"] == 0:
        model.set_train(False)
        cae_transformer_eval(model, eval_data, data_params)
        model.set_train(True)
print(f"=====================End cae transformer train========================")
cae_transformer_prediction(args)
plot_train_loss(train_loss, summary_dir, optimizer_params["epochs"], "cae")

====================Start cae transformer train=====================
epoch: 1 train loss: 1.6767721 epoch time: 20.05s
epoch: 2 train loss: 1.3125554 epoch time: 11.43s
epoch: 3 train loss: 1.0195727 epoch time: 11.36s
epoch: 4 train loss: 0.78560066 epoch time: 11.45s
epoch: 6 train loss: 0.45436704 epoch time: 11.63s
epoch: 7 train loss: 0.34071732 epoch time: 11.28s
epoch: 8 train loss: 0.25292587 epoch time: 11.78s
epoch: 9 train loss: 0.18576089 epoch time: 11.50s
epoch: 10 train loss: 0.13489997 epoch time: 11.36s
=================Start cae-transformer evaluation=====================
eval loss: 0.1344970128662193
===================End transformer evaluation====================
epoch: 11 train loss: 0.09680262 epoch time: 11.57s
epoch: 12 train loss: 0.06859333 epoch time: 11.35s
epoch: 13 train loss: 0.04796055 epoch time: 11.57s
epoch: 14 train loss: 0.03306444 epoch time: 11.67s
epoch: 15 train loss: 0.022457568 epoch time: 11.77s
epoch: 16 train loss: 0.0150149 epoch time: 10

### 预测结果

下为CAE-Transformer和真实值的对比：

<figure class="harf">
    <img src="./images/prediction_result.gif" title="prediction_result" width="500"/>
</figure>

结果展现了流场中不同位置的速度随时间的变化情况。预测结果与真实值的平均相对误差为6.3e-06。